In [5]:
!docker run \
  --name neo4j-test \
  -p 7474:7474 -p 7687:7687 \
  -e NEO4J_AUTH=neo4j/password \
  -d neo4j:5

aa5b1242d0100d108d7b8965f4ac832e1bc17bdb4596ef8e5f076efc4d1c37a0


↑ [WebUI](http://localhost:7474)でいろいろ見れるよ

---

In [7]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [8]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

def create_friend(tx, name, friend_name):
    tx.run("CREATE (a:Person {name: $name})-[:KNOWS]->(f:Person {name: $friend_name})",
           name=name, friend_name=friend_name)

def get_friends(tx, name):
    result = tx.run("MATCH (a:Person {name: $name})-[:KNOWS]->(f) RETURN f.name AS friend",
                    name=name)
    return [record["friend"] for record in result]

with driver.session() as session:
    session.execute_write(create_friend, "Alice", "Charlie")
    friends = session.execute_read(get_friends, "Alice")
    print(f"Alice knows: {friends}")

Alice knows: ['Charlie']


In [11]:
from neo4j import GraphDatabase

# 接続設定
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

# --- Create ---
def create_person(tx, name, age):
    tx.run("CREATE (p:Person {name: $name, age: $age})", name=name, age=age)

def create_relationship(tx, name1, name2):
    tx.run("""
        MATCH (a:Person {name: $name1}), (b:Person {name: $name2})
        CREATE (a)-[:KNOWS]->(b)
    """, name1=name1, name2=name2)

# --- Read ---
def get_people(tx):
    result = tx.run("MATCH (p:Person) RETURN p.name AS name, p.age AS age")
    return [dict(record) for record in result]

def get_friends(tx, name):
    result = tx.run("""
        MATCH (a:Person {name: $name})-[:KNOWS]->(f)
        RETURN f.name AS friend
    """, name=name)
    return [record["friend"] for record in result]

# --- Update ---
def update_age(tx, name, new_age):
    tx.run("MATCH (p:Person {name: $name}) SET p.age = $age", name=name, age=new_age)

# --- Delete ---
def delete_person(tx, name):
    tx.run("MATCH (p:Person {name: $name}) DETACH DELETE p", name=name)

# 実行例
with driver.session() as session:
    # Create
    session.execute_write(create_person, "Alice", 30)
    session.execute_write(create_person, "Bob", 25)
    session.execute_write(create_relationship, "Alice", "Bob")

    # Read
    people = session.execute_read(get_people)
    print("People:", people)
    print("Alice's friends:", session.execute_read(get_friends, "Alice"))

    # Update
    session.execute_write(update_age, "Alice", 31)
    print("After update:", session.execute_read(get_people))

    # Delete
    session.execute_write(delete_person, "Bob")
    print("After delete:", session.execute_read(get_people))

People: [{'name': 'Alice', 'age': 31}, {'name': 'Charlie', 'age': None}, {'name': 'Alice', 'age': 31}, {'name': 'Alice', 'age': 30}, {'name': 'Bob', 'age': 25}]
Alice's friends: ['Charlie', 'Bob', 'Bob', 'Bob']
After update: [{'name': 'Alice', 'age': 31}, {'name': 'Charlie', 'age': None}, {'name': 'Alice', 'age': 31}, {'name': 'Alice', 'age': 31}, {'name': 'Bob', 'age': 25}]
